In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Load cocoa prices
cocoa_df = pd.read_csv("../data/Daily Prices_ICCO.csv")
print("Cocoa data shape:", cocoa_df.shape)
print(cocoa_df.head())

# Load climate data
climate_df = pd.read_csv("../data/Ghana_data.csv")
print("\nGhana climate data shape:", climate_df.shape)
print(climate_df.head())

# Load currency exchange data
ghs_usd_df = pd.read_csv("../data/GHS_USD Historical Data.csv")
xof_usd_df = pd.read_csv("../data/USD_XOF Historical Data.csv")

print("GHS to USD data:", ghs_usd_df.shape)
print(ghs_usd_df.head())

print("\nUSD to XOF data:", xof_usd_df.shape)
print(xof_usd_df.head())

Cocoa data shape: (7812, 2)
         Date ICCO daily price (US$/tonne)
0  27/02/2025                     9,099.66
1  26/02/2025                     9,089.95
2  25/02/2025                     8,668.57
3  24/02/2025                     8,408.72
4  21/02/2025                     9,106.42

Ghana climate data shape: (53231, 7)
       STATION                      NAME        DATE  PRCP  TAVG  TMAX  TMIN
0  GHM00065472  KOTOKA INTERNATIONAL, GH  1990-01-01   NaN    80  91.0  76.0
1  GHM00065472  KOTOKA INTERNATIONAL, GH  1990-01-02   NaN    81  92.0   NaN
2  GHM00065472  KOTOKA INTERNATIONAL, GH  1990-01-03   NaN    82   NaN   NaN
3  GHM00065472  KOTOKA INTERNATIONAL, GH  1990-01-04   NaN    81   NaN  75.0
4  GHM00065472  KOTOKA INTERNATIONAL, GH  1990-01-05   0.0    79  89.0  75.0
GHS to USD data: (4448, 7)
         Date   Price    Open    High     Low Vol. Change %
0  03/21/2025  0.0643  0.0645  0.0645  0.0644  NaN    0.00%
1  03/20/2025  0.0643  0.0645  0.0646  0.0643  NaN    0.00%
2  03/1

In [23]:
# Cleaning cocoa price data
cocoa_df_clean = cocoa_df.copy()
cocoa_df_clean.columns = ['Date', 'Price']
cocoa_df_clean['Date'] = pd.to_datetime(cocoa_df_clean['Date'], format='%d/%m/%Y')
cocoa_df_clean['Price'] = cocoa_df_clean['Price'].str.replace(',', '').astype(float)
cocoa_df_clean = cocoa_df_clean.sort_values('Date').reset_index(drop=True)

# Cleaning GHS/USD exchange rate
ghs_usd_clean = ghs_usd_df[['Date', 'Price']].copy()
ghs_usd_clean.columns = ['Date', 'GHS_USD']
ghs_usd_clean['Date'] = pd.to_datetime(ghs_usd_clean['Date'], format='%m/%d/%Y')
ghs_usd_clean['GHS_USD'] = ghs_usd_clean['GHS_USD'].astype(float)
ghs_usd_clean = ghs_usd_clean.sort_values('Date').reset_index(drop=True)

# Cleaning USD/XOF exchange rate
xof_usd_clean = xof_usd_df[['Date', 'Price']].copy()
xof_usd_clean.columns = ['Date', 'USD_XOF']
xof_usd_clean['Date'] = pd.to_datetime(xof_usd_clean['Date'], format='%Y-%m-%d')
xof_usd_clean['USD_XOF'] = xof_usd_clean['USD_XOF'].astype(float)
xof_usd_clean = xof_usd_clean.sort_values('Date').reset_index(drop=True)

# Checking cleaned versions
cocoa_df_clean.head(), ghs_usd_clean.head(), xof_usd_clean.head()

# Convert Fahrenheit to Celsius for the climate data
climate_df_clean = climate_df.copy()
climate_df_clean['DATE'] = pd.to_datetime(climate_df_clean['DATE'])

# Convert temperature columns (Fahrenheit to Celsius)
for col in ['TAVG', 'TMAX', 'TMIN']:
    climate_df_clean[col] = (climate_df_clean[col] - 32) * 5.0/9.0

# Select only relevant columns and rename
climate_df_clean = climate_df_clean[['DATE', 'NAME', 'PRCP', 'TAVG', 'TMAX', 'TMIN']].copy()
climate_df_clean.columns = ['Date', 'Station_Name', 'PRCP', 'TAVG_C', 'TMAX_C', 'TMIN_C']


# Check cleaned climate data
climate_df_clean.head()


,Date,Station_Name,PRCP,TAVG_C,TMAX_C,TMIN_C
0,1990-01-01,"KOTOKA INTERNATIONAL, GH",NaN,26.666667,32.777778,24.444444
1,1990-01-02,"KOTOKA INTERNATIONAL, GH",NaN,27.222222,33.333333,NaN
2,1990-01-03,"KOTOKA INTERNATIONAL, GH",NaN,27.777778,NaN,NaN
3,1990-01-04,"KOTOKA INTERNATIONAL, GH",NaN,27.222222,NaN,23.888889
4,1990-01-05,"KOTOKA INTERNATIONAL, GH",0.0,26.111111,31.666667,23.888889


In [24]:
# Merge datasets step-by-step on the Date column
merged_df = cocoa_df_clean.merge(ghs_usd_clean, on='Date', how='left')
merged_df = merged_df.merge(xof_usd_clean, on='Date', how='left')
merged_df = merged_df.merge(climate_df_clean, on='Date', how='left')

# Preview the merged result
merged_df.head()


,Date,Price,GHS_USD,USD_XOF,Station_Name,PRCP,TAVG_C,TMAX_C,TMIN_C
0,1994-10-03,1466.24,NaN,NaN,"KOTOKA INTERNATIONAL, GH",0.00,25.000000,NaN,22.777778
1,1994-10-04,1462.28,NaN,NaN,"KOTOKA INTERNATIONAL, GH",0.00,27.222222,NaN,NaN
2,1994-10-05,1430.57,NaN,NaN,"KOTOKA INTERNATIONAL, GH",0.24,26.111111,NaN,NaN
3,1994-10-05,1430.57,NaN,NaN,"SEFWI BEKWAI, GH",NaN,27.777778,NaN,NaN
4,1994-10-06,1430.32,NaN,NaN,"KOTOKA INTERNATIONAL, GH",0.01,26.666667,NaN,22.777778


In [25]:
merged_post_2008 = merged_df[merged_df['Date'] >= pd.to_datetime("2008-01-01")].copy()

# Calculate split indices for 80-10-10 split
total_rows = len(merged_post_2008)
train_end = int(total_rows * 0.8)
val_end = int(total_rows * 0.9)

# Split the data
train_df = merged_post_2008.iloc[:train_end].copy()
val_df = merged_post_2008.iloc[train_end:val_end].copy()
test_df = merged_post_2008.iloc[val_end:].copy()

# Show the number of rows in each set and their date ranges
(train_df.shape[0], train_df['Date'].min(), train_df['Date'].max()), \
(val_df.shape[0], val_df['Date'].min(), val_df['Date'].max()), \
(test_df.shape[0], test_df['Date'].min(), test_df['Date'].max())


((23426, Timestamp('2008-01-02 00:00:00'), Timestamp('2022-04-22 00:00:00')),
 (2928, Timestamp('2022-04-22 00:00:00'), Timestamp('2023-09-22 00:00:00')),
 (2929, Timestamp('2023-09-22 00:00:00'), Timestamp('2025-02-27 00:00:00')))

In [26]:
train_df.to_csv("../data/train.csv", index=False)
val_df.to_csv("../data/val.csv", index=False)
test_df.to_csv("../data/test.csv", index=False)
merged_post_2008.to_csv("../data/merged_dataset.csv", index=False)


In [3]:
ghs_df = pd.read_csv('../data/GHS_USD Historical Data.csv')
print(ghs_df["Date"].head())  # Show first 5 date values

0    03/21/2025
1    03/20/2025
2    03/19/2025
3    03/18/2025
4    03/17/2025
Name: Date, dtype: object
